In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Define the dataset path
dataset_path = "/kaggle/input/plant-leave-diseases-dataset-with-augmentation"

# Define transformations, including scaling down image pixel values
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x / 256)  # Normalizing the images
])

# Load the dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Splitting the dataset into train and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Custom DataLoader
def custom_data_loader(dataset, batch_size, shuffle=True):
    indices = list(range(len(dataset)))
    if shuffle:
        np.random.shuffle(indices)
    for start_idx in range(0, len(dataset), batch_size):
        batch_indices = indices[start_idx:start_idx + batch_size]
        batch = [dataset[i] for i in batch_indices]
        images, labels = zip(*batch)
        yield torch.stack(images), torch.tensor(labels)

# Display function for datasets
def show_images(images, labels, predictions=None):
    plt.figure(figsize=(10, 5))
    for i in range(min(len(images), 5)):
        plt.subplot(1, 5, i + 1)
        plt.imshow(np.transpose(images[i], (1, 2, 0)))
        title = f"Label: {labels[i]}"
        if predictions is not None:
            title += f"\nPred: {predictions[i]}"
        plt.title(title)
        plt.axis('off')
    plt.show()

# Define the CNN model
class MultiClassCNN(nn.Module):
    def __init__(self):
        super(MultiClassCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 127 * 127, 39)  # Adjust according to the number of classes

    def forward(self, x):
        x = self.conv_block(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Initializing the model, loss function, and optimizer
model = MultiClassCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(train_loader, model, criterion, optimizer, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Evaluate the model
def evaluate_model(model, data_loader):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())
    
    print(classification_report(all_labels, all_preds))
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=dataset.classes, yticklabels=dataset.classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Prepare DataLoaders
train_loader = custom_data_loader(train_dataset, batch_size=32)
test_loader = custom_data_loader(test_dataset, batch_size=32)

# Training the model
train_model(train_loader, model, criterion, optimizer)

# Evaluating the model
evaluate_model(model, test_loader)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
!pip install torchmetrics

In [ ]:
!pip install torchvision

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.datasets as datasets
from torchmetrics import Accuracy, Precision, Recall

In [ ]:
# Define the dataset path
dataset_path = 'C:/Users/Babacar Gaye/Desktop/mes docs babs/mes etudes/ESSAI 2EM ANNE/PFA/Plant_leave_diseases_dataset_with_augmentation'

# Define the transformations
transform = transforms.Compose([ transforms.Resize((256, 256)),transforms.ToTensor(),          
])

# Load the dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [ ]:
# Define the split ratio (e.g., 80% for training, 20% for testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
# Use the Subset class to split the dataset
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# Define batch size for DataLoader
batch_size = 32

# Create DataLoader instances for training and testing
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:

# Define a function to display images from a DataLoader
def show_images(dataloader, num_images=5):
    # Get a batch of images
    for images, labels in dataloader:
        # Plot the images
        plt.figure(figsize=(10, 5))
        for i in range(num_images):
            plt.subplot(1, num_images, i + 1)
            plt.imshow(np.transpose(images[i], (1, 2, 0)))  # Transpose the image tensor to (height, width, channels)
            plt.title(f"Label: {labels[i]}")
            plt.axis('off')
        plt.show()
        break  # Break after showing one batch

# Display images from the training DataLoader
show_images(train_loader)

In [ ]:
# Obtenez l'ensemble de données parent de votre sous-ensemble de données
parent_dataset = train_data.dataset

# Obtenez le nombre de classes à partir de l'attribut 'classes' de l'ensemble de données parent
num_classes = len(parent_dataset.classes)
print("Nombre de classes :", num_classes)

In [ ]:
class MultiClassCNN(nn.Module):
    def __init__(self):
        super(MultiClassCNN,self).__init__()
        self.conv_block = nn.Sequential(nn.Conv2d(3,16,kernel_size=3, stride=1),nn.ReLU(),nn.MaxPool2d(kernel_size=2, stride=2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16*127*127, 39)
    def forward(self,x):
        x=self.conv_block(x)
        x=self.flatten(x)
        x=self.fc(x)
        return x

In [ ]:
# Initialisation du modèle
model = MultiClassCNN()

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Boucle d'apprentissage
for epoch in range(2):
    running_loss = 0.0
    for images, labels in train_loader:
        # Remise à zéro des gradients
        optimizer.zero_grad()
        
        # Passage avant (forward pass) pour obtenir les prédictions du modèle
        outputs = model(images)
        
        # Calcul de la perte
        loss = criterion(outputs, labels)
        
        # Rétropropagation des gradients
        loss.backward()
        
        # Mise à jour des poids du modèle
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")


In [ ]:
predictions = []

# Mettez le modèle en mode d'évaluation
model.eval()

# Boucle à travers les données de test
for images, labels in test_data:
    # Passez les images à travers le modèle pour obtenir les prédictions
    with torch.no_grad():  # Pas de calcul de gradient pendant l'évaluation
        outputs = model(images.unsqueeze(0))  # Ajoutez une dimension de lot (batch) supplémentaire
    predicted_label = torch.argmax(outputs, dim=1).item()  # Convertit les sorties en étiquettes prédites
    predictions.append(predicted_label)  # Ajoutez l'étiquette prédite à la liste des prédictions

# Affichez les 10 premières prédictions
print("First 10 predictions:", predictions[:10])


In [ ]:

# Initialize metrics
accuracy = Accuracy(task='multiclass',num_classes=39, average='macro')
precision = Precision(task='multiclass',num_classes=39, average=None)
recall = Recall(task='multiclass',num_classes=39, average=None)

# Put the model in evaluation mode
model.eval()

# Iterate over the test data
for images, labels in test_loader:
    # Forward pass to get the predicted labels
    with torch.no_grad():
        outputs = model(images)
    predicted = torch.argmax(outputs, dim=1)
    
    # Update metrics
    accuracy.update(predicted, labels)
    precision.update(predicted, labels)
    recall.update(predicted, labels)

# Compute final values
accuracy = accuracy.compute()
precision = precision.compute()
recall = recall.compute()

# Print accuracy, precision, and recall
print("Accuracy:", accuracy)
print("Per-class Precision:", precision)
print("Per-class Recall:", recall)
